# Model Training

## Prerequisites

To run this notebook the `data_path` variable has to point to the location of a valid dataset. If you don't have any data an example dataset can be obtained by characterizing [PTM](http://ptm.asu.edu/) transistor models as shown in [pyrdict](https://github.com/AugustUnderground/pyrdict).

Additionally, for training [precept](https://github.com/electronics-and-drives/precept) has to be installed manually.

## Setup

In [1]:
import os
import torch
import h5py as h5
import numpy as np
import torch as pt
import pandas as pd
import joblib as jl
from datetime import datetime as dt
from functools import partial
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler, minmax_scale

In [2]:
from pytorch_lightning import Trainer
from precept import PreceptModule, PreceptDataFrameModule

In [3]:
%matplotlib notebook

In [4]:
num_gpus = pt.cuda.device_count() if pt.cuda.is_available() else None

## Data

**Note**: This part has to be adjusted according to the location and shape of custom datasets.

- Adjust `data_path` accordingly, so it points to a your $\frac{g_{\mathrm{m}}}{I_{\mathrm{d}}}$ look up table.
- Depending on the shape of the dataset and file format it may need to be read differently.

If the data is generated with [pyrdict](https://github.com/AugustUnderground/pyrdict) the resulting `h5` is structured such that each parameter is a group.

In [5]:
device_type = "nmos"
device_tech = "90nm"
device_name = "ptm"
data_path   = f"../data/{device_tech}_bulk_{device_type}.h5"

In [6]:
with h5.File(data_path, "r") as data_base:
    column_names = list(data_base.keys())
    data_matrix  = np.transpose(np.array([data_base[c] for c in column_names]))
    data_frame   = pd.DataFrame(data_matrix, columns=column_names).dropna()

### Preprocesing and Filtering

Only the data necessary for preprocessing and the function mappings is kept, everything else can be filtered out.

In [7]:
params   = [ "Vgs", "Vds", "Vbs", "L", "W"
           , "gmid", "fug", "vdsat", "a0"
           , "jd", "id", "gm", "gds" ]
data_raw = data_frame[params]

Since in the vast majority of cases a device is desired to be in saturation all other cases are filtered out.

In [8]:
data_filtered = data_raw[( (round(data_raw.Vgs, 2) == round(data_raw.Vds, 2))
                         & (round(data_raw.Vbs, 2) == 0.0)
                         & (data_raw.W == np.random.choice(data_raw.W.unique())))]
data_filtered.describe()

,Vgs,Vds,Vbs,L,W,gmid,fug,vdsat,a0,jd,id,gm,gds
count,1200.000000,1200.000000,1.200000e+03,1.200000e+03,1200.000000,1200.000000,1.200000e+03,1200.000000,1200.000000,1200.000000,1.200000e+03,1.200000e+03,1.200000e+03
mean,0.604988,0.605000,3.717822e-09,5.075000e-06,0.000075,11.808869,6.372252e+09,0.252597,308.244902,30.798201,2.309865e-03,6.531383e-03,1.533531e-04
std,0.346527,0.346543,1.368803e-08,3.144863e-06,0.000000,9.293653,2.246506e+10,0.202448,186.647671,97.946921,7.346019e-03,1.630982e-02,5.800625e-04
min,0.010000,0.010000,2.476312e-14,1.500000e-07,0.000075,1.402050,3.244685e+02,0.041947,0.312317,0.000003,2.539053e-10,6.790613e-09,3.972947e-09
25%,0.307500,0.307500,6.023953e-13,2.338889e-06,0.000075,3.105885,7.796667e+06,0.049042,164.463758,0.049760,3.731991e-06,7.985212e-05,2.196382e-07
50%,0.604999,0.605000,1.946944e-12,5.075000e-06,0.000075,7.858949,3.909210e+07,0.204439,317.307761,3.725840,2.794380e-04,1.729884e-03,4.108271e-06
75%,0.902486,0.902500,1.963356e-10,7.811111e-06,0.000075,21.654458,1.896120e+08,0.424186,447.174792,15.575919,1.168194e-03,3.759085e-03,1.358573e-05
max,1.199914,1.200000,1.379620e-07,1.000000e-05,0.000075,26.747242,9.754683e+10,0.671173,675.272394,718.132061,5.385990e-02,7.557484e-02,3.434507e-03


### Mappings

For each device ($\delta \in [\mathrm{"NMOS"}, \mathrm{"PMOS"}]$) and 
technology ($\tau = 90\,\mathrm{nm}$ in this case) two mappings ($\gamma, \nu$) are trained,
where

$$\gamma_{\mathrm{\delta,\tau}} = 
    \begin{bmatrix} 
        \frac{g_{\mathrm{m}}}{I_{\mathrm{d}}} \\ f_{\mathrm{ug}} 
    \end{bmatrix} 
    \mapsto
    \begin{bmatrix}
        J_{\mathrm{d}} \\ L
    \end{bmatrix}$$

and
    
$$\nu_{\delta,\tau} = 
    \begin{bmatrix} 
        v_{\mathrm{d,sat}} \\ f_{\mathrm{ug}} 
    \end{bmatrix} 
    \mapsto
    \begin{bmatrix}
        J_{\mathrm{d}} \\ L
    \end{bmatrix}$$

For this the following `mappings` are defined

In [14]:
mappings = { "γ": { "x": ["gmid", "fug"]
                  , "y": ["jd", "L", "a0"] 
                  , }
           , "ν": { "x": ["vdsat", "fug"]
                  , "y": ["jd", "L", "a0"]
                  , }
           , }

## Training Setup

The batch size is deliberatly chosen to be small, based on experience with other datasets but may be increased at will. Same goes for the number of epochs.

In [15]:
batch_size = 2000
test_split = 0.2
num_epochs = 42

A folder for storing the model will be dynamically created in the root of this repositry each time the notebook is run.
All trained models will be located under `models/custom/` and assigned with a timestamp.

In [16]:
time_stamp = dt.now().strftime("%Y-%m%d-%H%M%S")
model_dir  = f"../models/custom/{device_tech}-{device_name}-{time_stamp}/"
os.makedirs(model_dir, exist_ok = True)

The `trainingSetup` function is purely for convenience, and returns _precept_ modules for the lightning trainer.

In [17]:
def trainingSetup(data, params_x, params_y, model_path):
    
    data = PreceptDataFrameModule( data                    # unecessary columns are filtered out internally
                                 , params_x, params_y      # input and output parameters
                                 , [], [], [], []          # No transformations
                                 , batch_size = batch_size
                                 , test_split = test_split
                                 , scale      = False      # No need to scale data, we did that before
                                 , )
    
    module = PreceptModule( len(params_x), len(params_y)
                          , model_path = model_path
                          , )
    
    return (module, data)

In [18]:
for mapping, params in mappings.items():
    print(f"Training {device_name}: {mapping}({params['x']}) -> ({params['y']})")
    
    model_pre   = f"{model_dir}/{'v' if mapping == 'ν' else 'g'}-{device_type}-{device_tech}"
    
    scale_x     = MinMaxScaler()
    scale_y     = MinMaxScaler()
    
    scaled_x    = scale_x.fit_transform(data_filtered[params['x']].values).T
    scaled_y    = scale_y.fit_transform(data_filtered[params['y']].values).T
    
    data_scaled = pd.DataFrame( np.vstack((scaled_x, scaled_y)).T
                               , columns = (params['x'] + params['y']) )
    
    module,data = trainingSetup(data_scaled, params['x'], params['y'], model_dir)
    
    trainer     = Trainer( gpus                = num_gpus
                         , max_epochs          = num_epochs
                         , precision           = 64
                         , checkpoint_callback = True
                         , default_root_dir    = model_dir
                         , )
    
    trainer.fit(module, data)
    trainer.save_checkpoint(f"{model_pre}.cktp")
    
    jl.dump(scale_x, f"{model_pre}.X")
    jl.dump(scale_y, f"{model_pre}.Y")

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: ModelCheckpoint
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 1.4 M 
------------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
11.038    Total estimated model params size (MB)


Training ptm: γ(['gmid', 'fug']) -> (['jd', 'L', 'a0'])


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]